In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(1, '/home/jw3514/Work/ASD_Circuits/src')
from ASD_Circuits import *
from tabulate import tabulate
import csv

In [ ]:
def searchFil(text, DIR):
    RES = []
    for file in os.listdir(DIR):
        if text in file:
            RES.append(file)
    return RES

def LoadSA2(fname, DIR, topL=50):
    fin = open(DIR+fname, 'rt')
    max_cohe, max_bias, max_STRs = 0, 0, []
    for i, l in enumerate(fin):
        if i > topL:
            break
        l = l.strip().split()
        cohe = float(l[0])
        bias = float(l[1])
        STRs = l[2].split(",")
        if cohe > max_cohe:
            max_cohe = cohe
            max_bias = bias
            max_STRs = STRs
    return max_cohe, max_bias, max_STRs

In [ ]:
InfoMat = pd.read_csv("../dat/allen-mouse-conn/ScoreingMat_jw_v3/InfoMat.Ipsi.csv", index_col=0)
adj_mat = pd.read_csv("../dat/allen-mouse-conn/ScoreingMat_jw_v3/WeightMat.Ipsi.csv", index_col=0)

In [ ]:
size = 46
#biaslim_df = pd.read_csv(biaslim_dir + "biaslim.size.{}.txt".format(size), names=["size", "bias"])
ASD_DIR_SI = "/home/jw3514/Work/ASD_Circuits/dat/Circuits/SA/ASD_Pareto_Dec14_Info/"
ASD_DIR_Conn = "/home/jw3514/Work/ASD_Circuits/dat/Circuits/SA/ASD_Pareto_Dec14_Conn/"
ASD_BiasDF = pd.read_csv("../dat/Unionize_bias/Spark_Meta_EWS.Z2.bias.csv", index_col="STR")
biaslim_df = pd.read_csv(
    "../dat/Circuits/SA/biaslims2/biaslim.size.46.top17.txt", names=["size", "bias"])
biaslim_df = biaslim_df.sort_values("bias", ascending=False)

In [ ]:
f = open("ASD.SA.Circuits.Size46.csv", 'wt')
writer = csv.writer(f) 
writer.writerow(["idx", "Bias", "SI_score", "Conn_score", "gain STRs", "lost STRs", "STRs"])

In [ ]:
idx = 0
top_STRs = ASD_BiasDF.head(size).index.values
scoreSI = ScoreCircuit_SI_Joint(top_STRs, InfoMat)
scoreCO = ScoreCircuit_NEdges(top_STRs, adj_mat)
print(scoreSI, scoreCO)
print(RegionDistributionsList(top_STRs))
writer.writerow([idx, ASD_BiasDF.loc[top_STRs, "EFFECT"].mean(), scoreSI, scoreCO, 
                "", "", ";".join(top_STRs)])


In [ ]:
cc = 0
PrevSet = set(top_STRs)
for i, row in biaslim_df.iterrows():
    #print(row["fname"], DIR)
    idx += 1
    fil = searchFil("keepN_{}-minbias_{}.txt".format(size, row["bias"]), ASD_DIR_SI)[0]
    cohe, real_minbias, STRs = LoadSA2(fil, ASD_DIR_SI)
    score = ScoreCircuit_SI_Joint(STRs, InfoMat)
    score2 = ScoreCircuit_NEdges(STRs, adj_mat)
    add_strs = list(set(STRs).difference(PrevSet))
    lost_strs = list(set(PrevSet).difference(STRs))
    print(row["bias"], round(real_minbias, 3), round(score, 3),
          len(set(STRs).difference(top_STRs)))
    print("add", add_strs)
    print("drop", lost_strs)
    PrevSet = set(STRs)
    cc += 1
    if cc > 16:
        break
        
    if row["bias"] == 0.37:
        List2Fil(STRs, "Circuits.Info.Size46.Bias0.37.txt")
    if row["bias"] == 0.36:
        List2Fil(STRs, "Circuits.Info.Size46.Bias0.36.txt")
    
    writer.writerow([idx, real_minbias, score, score2, 
                ";".join(add_strs), ";".join(lost_strs), ";".join(STRs)])
    print()
f.close()

In [ ]:
cc = 0
PrevSet = set(top_STRs)
for i, row in biaslim_df.iterrows():
    #print(row["fname"], DIR)

    fil = searchFil("keepN_{}-minbias_{}.txt".format(size, row["bias"]), ASD_DIR_Conn)[0]
    cohe, real_minbias, STRs = LoadSA2(fil, ASD_DIR_Conn)
    score = ScoreCircuit_NEdges(STRs, adj_mat)
    print(row["bias"], round(real_minbias, 3), round(score, 3),
          len(set(STRs).difference(top_STRs)))
    print("add", set(STRs).difference(PrevSet))
    print("drop", set(PrevSet).difference(STRs))
    PrevSet = set(STRs)
    cc += 1
    if cc > 16:
        break
        
    if row["bias"] == 0.37:
        List2Fil(STRs, "Circuits.Info.Size46.Bias0.37.txt")
    if row["bias"] == 0.36:
        List2Fil(STRs, "Circuits.Info.Size46.Bias0.36.txt")
        
    print()